In [7]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import pandas as pd
import util as util
import data_pipeline as data_pipeline
import preprocessing as preprocessing

ModuleNotFoundError: No module named 'util'

In [ ]:
config_data = util.load_config()
ohe_ContractRenewal = util.pickle_load(config_data["ohe_ContractRenewal_path"])
ohe_DataPlan = util.pickle_load(config_data["ohe_DataPlan_path"])
le_encoder = util.pickle_load(config_data["le_encoder_path"])
model_data = util.pickle_load(config_data["production_model_path"])

NameError: name 'util' is not defined

In [10]:
class api_data(BaseModel):
    AccountWeeks : int   
    ContractRenewal : str
    DataPlan : str
    DataUsage : float
    CustServCalls : int
    DayMins : float
    DayCalls : int
    MonthlyCharge : float
    OverageFee : float
    RoamMins : float

In [12]:
app = FastAPI()

In [13]:
@app.get("/")
def home():
    return "Hello, FastAPI up!"

In [14]:
def predict(data: api_data):    
    # Convert data api to dataframe
    data = pd.DataFrame(data).set_index(0).T.reset_index(drop = True)

    # Convert dtype
    data = pd.concat(
        [
            data[config_data["predictors"][0]],
            data[config_data["predictors"][1:]].astype(int)
        ],
        axis = 1
    )

    # Check range data
    try:
        data_pipeline.check_data(data, config_data, True)
    except AssertionError as ae:
        return {"res": [], "error_msg": str(ae)}
    
    # Encoding ContractRenewal
    data = preprocessing.ohe_transform_ContractRenewal(data, "ContractRenewal", ohe_ContractRenewal)
    
    #Cek lagi
    # Encoding DataPlan
    data = preprocessing.ohe_transform_DataPlan(data, "DataPlan", ohe_DataPlan)

    # Predict data
    y_pred = model_data["model_data"]["model_object"].predict(data)

    # Inverse tranform
    y_pred = list(le_encoder.inverse_transform(y_pred))[0] 

    return {"res" : y_pred, "error_msg": ""}


In [15]:
uvicorn.run("api:app", host = "0.0.0.0", port = 8080)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
data = pd.DataFrame(data).set_index(0).T.reset_index(drop = True)

NameError: name 'data' is not defined

In [17]:
log = pd.read_json("training_log.json")

ValueError: Unexpected character found when decoding 'true'

In [18]:
log['performance']

NameError: name 'log' is not defined

In [ ]:
log["performance"][0]

[{'0': {'precision': 0.9384615384615381,
   'recall': 0.7142857142857141,
   'f1-score': 0.811170212765957,
   'support': 427},
  '1': {'precision': 0.30285714285714205,
   'recall': 0.7260273972602741,
   'f1-score': 0.427419354838709,
   'support': 73},
  'accuracy': 0.716,
  'macro avg': {'precision': 0.62065934065934,
   'recall': 0.7201565557729941,
   'f1-score': 0.619294783802333,
   'support': 500},
  'weighted avg': {'precision': 0.8456632967032961,
   'recall': 0.716,
   'f1-score': 0.7551425875085791,
   'support': 500}},
 {'0': {'precision': 0.9463276836158191,
   'recall': 0.7845433255269321,
   'f1-score': 0.8578745198463501,
   'support': 427},
  '1': {'precision': 0.36986301369863,
   'recall': 0.73972602739726,
   'f1-score': 0.49315068493150604,
   'support': 73},
  'accuracy': 0.778,
  'macro avg': {'precision': 0.6580953486572241,
   'recall': 0.7621346764620961,
   'f1-score': 0.6755126023889281,
   'support': 500},
  'weighted avg': {'precision': 0.862163841807909